In [1]:
import sys
import os
import requests
import datetime
from pathlib import Path
#
path_root = Path(os.path.abspath(''))
sys.path.insert(1, os.path.join(path_root))
#
path_root2 = Path(os.path.abspath('')).parent
sys.path.insert(1, os.path.join(path_root2))

import time
import talib
import sqlite3 as sqlite3
import yfinance as yf
import pandas as pd
from util.util import get_ucodes

path_sqlite = os.path.join(path_root, 'data', 'aastock', 'sqlite', 'hk-marketwatch.db')
if not os.path.exists(path_sqlite):
    path_sqlite = os.path.join(path_root2, 'data', 'aastock', 'sqlite', 'hk-marketwatch.db')

In [2]:
conn = sqlite3.connect(path_sqlite)
cursor = conn.cursor()

ucodes = get_ucodes()
data = {}
ucodes1 = ['2800.HK', '2828.HK', '2823.HK', 'spy', 'vti', 'qqq', 'shy', 'tlt'] # '3188.HK', 'voo', 'tqqq'
ucodes2 = ['hsi', 'hsce', 'ssec', 'csi300', 'sp500', 'ixic', 'shy', 'tlt']
for ucode in ucodes2:
    if '.HK' in ucode:
        ucode = ucode.replace('.HK', '').zfill(5)
    
    #
    sql = """SELECT t.code, t.lot, t.nmll, t.stime, t.high, t.low, t.open, t.close, t.volume
                FROM (SELECT n.code, n.lot, n.nmll, c.stime, c.high, c.low, c.open, c.close, c.volume 
                    FROM s_{} AS c INNER JOIN name AS n 
                        ON c.code=n.code ORDER BY c.stime DESC) AS t 
                            ORDER BY t.stime """.format(ucode)
    cursor.execute(sql)
    columns = ['code', 'lot', 'nmll', 'sdate', 'high', 'low', 'open', 'last', 'vol']
    data[ucode] = pd.DataFrame(cursor.fetchall(), columns=columns)
    data[ucode].index = pd.to_datetime(data[ucode].sdate)
    print(f"{ucode}:   {data[ucode].index[0]} ~ {data[ucode].index[-1]}  {data[ucode].shape}")
conn.close()

hsi:   1986-12-31 00:00:00 ~ 2021-08-02 00:00:00  (8504, 9)
hsce:   1993-07-15 00:00:00 ~ 2021-08-02 00:00:00  (6867, 9)
ssec:   1997-07-02 00:00:00 ~ 2021-08-02 00:00:00  (5837, 9)
csi300:   2004-12-31 00:00:00 ~ 2021-08-02 00:00:00  (4078, 9)
sp500:   1927-12-30 00:00:00 ~ 2021-08-02 00:00:00  (19652, 9)
ixic:   1971-02-05 00:00:00 ~ 2021-08-02 00:00:00  (12369, 9)
shy:   2002-07-30 00:00:00 ~ 2021-08-02 00:00:00  (4786, 9)
tlt:   2002-07-30 00:00:00 ~ 2021-08-02 00:00:00  (4786, 9)


In [3]:
data_all = []
nmll = []
for k, df in data.items():
    data_all.append(df['last'])
    nmll.append(k)
df2 = pd.concat(data_all, axis=1)
df2 = df2.dropna()
df2.columns = nmll
print(f"{df2.index[0]} ~ {df2.index[-1]}  {df2.shape}")

2005-01-04 00:00:00 ~ 2021-08-02 00:00:00  (3786, 8)


In [4]:
# 相關系數
df2.corr()

,hsi,hsce,ssec,csi300,sp500,ixic,shy,tlt
hsi,1.000000,0.712951,0.668874,0.799502,0.741825,0.696657,0.679330,0.630135
hsce,0.712951,1.000000,0.797357,0.730379,0.149010,0.135673,0.402767,0.143673
ssec,0.668874,0.797357,1.000000,0.918009,0.355425,0.334053,0.379730,0.293475
csi300,0.799502,0.730379,0.918009,1.000000,0.634804,0.641807,0.638353,0.581330
sp500,0.741825,0.149010,0.355425,0.634804,1.000000,0.986186,0.707096,0.866956
ixic,0.696657,0.135673,0.334053,0.641807,0.986186,1.000000,0.732378,0.875126
shy,0.679330,0.402767,0.379730,0.638353,0.707096,0.732378,1.000000,0.890716
tlt,0.630135,0.143673,0.293475,0.581330,0.866956,0.875126,0.890716,1.000000


In [5]:
start_year = 2005

In [6]:
# 年度
df3_1 = df2.copy(deep=True)
df3 = pd.DataFrame(columns=['Date']+df3_1.columns.to_list())
for year in range(start_year, 2022, 1):
    df3_2 = df3_1[f'{year}-01-01': f'{year}-12-31'].iloc[[0, -1]].pct_change()*100
    df3.loc[df3.shape[0]+1] = [f'{year}-01-01 ~ {year}-12-31']+df3_2.iloc[-1].to_list()
print('年度標準差(風險)')
print(df3.std())
print('年度收益率(回報)')
df3

年度標準差(風險)
hsi       22.880302
hsce      32.421828
ssec      47.455654
csi300    56.278571
sp500     15.270059
ixic      19.824216
shy        2.115475
tlt       14.272171
dtype: float64
年度收益率(回報)


,Date,hsi,hsce,ssec,csi300,sp500,ixic,shy,tlt
1,2005-01-01 ~ 2005-12-31,5.912966,13.465805,-6.575374,-6.038193,5.070493,4.623645,1.742775,9.513994
2,2006-01-01 ~ 2006-12-31,31.346332,85.644138,126.550205,116.803303,11.373745,6.707876,3.648468,0.598894
3,2007-01-01 ~ 2007-12-31,36.678186,54.541849,93.744750,158.250479,4.240875,9.009021,6.968812,8.208596
4,2008-01-01 ~ 2008-12-31,-47.796773,-50.697234,-65.468058,-66.245362,-37.584650,-39.568824,6.428386,32.050025
5,2009-01-01 ~ 2009-12-31,38.123701,44.417778,73.476495,89.003425,21.453451,40.739421,0.806951,-17.273397
6,2010-01-01 ~ 2010-12-31,5.389018,-1.283944,-14.926661,-13.326746,11.023047,15.359426,2.022522,7.915778
7,2011-01-01 ~ 2011-12-31,-22.114178,-23.661663,-22.899109,-26.458437,-0.991968,-2.838232,1.488841,34.860822
8,2012-01-01 ~ 2012-12-31,21.034944,12.718228,2.943780,7.886711,9.796446,11.778985,0.297714,7.070603
9,2013-01-01 ~ 2013-12-31,-0.105864,-9.393217,-7.071343,-7.700139,26.041448,34.656602,0.226483,-11.347533
10,2014-01-01 ~ 2014-12-31,1.135337,11.908769,53.346781,52.185120,12.386594,14.312575,0.399435,26.915664


In [7]:
# 累計 年化收益率
df4 = pd.DataFrame(columns=['Date']+df3_1.columns.to_list())
df4['Date'] = df3['Date']
for code in data.keys():
    annualized_total_return = 1
    annualized_total_returns = []
    for v in df3[code]:
        no_year = df3[code].shape[0]
        annualized_total_return = annualized_total_return * (1+(v/100))
        annualized_total_returns.append(annualized_total_return)
    df4[code] = annualized_total_returns

print('累計 年化收益率')
df4

累計 年化收益率


,Date,hsi,hsce,ssec,csi300,sp500,ixic,shy,tlt
1,2005-01-01 ~ 2005-12-31,1.059130,1.134658,0.934246,0.939618,1.050705,1.046236,1.017428,1.095140
2,2006-01-01 ~ 2006-12-31,1.391128,2.106426,2.116537,2.037123,1.170209,1.116417,1.054548,1.101699
3,2007-01-01 ~ 2007-12-31,1.901368,3.255310,4.100679,5.260880,1.219837,1.216995,1.128038,1.192133
4,2008-01-01 ~ 2008-12-31,0.992576,1.604958,1.416044,1.775791,0.761365,0.735444,1.200552,1.574211
5,2009-01-01 ~ 2009-12-31,1.370982,2.317844,2.456504,3.356306,0.924704,1.035060,1.210240,1.302292
6,2010-01-01 ~ 2010-12-31,1.444865,2.288085,2.089830,2.909019,1.026635,1.194039,1.234718,1.405378
7,2011-01-01 ~ 2011-12-31,1.125345,1.746686,1.611277,2.139338,1.016451,1.160150,1.253101,1.895305
8,2012-01-01 ~ 2012-12-31,1.362060,1.968833,1.658710,2.308062,1.116027,1.296804,1.256831,2.029314
9,2013-01-01 ~ 2013-12-31,1.360619,1.783896,1.541417,2.130338,1.406657,1.746232,1.259678,1.799037
10,2014-01-01 ~ 2014-12-31,1.376066,1.996337,2.363713,3.242057,1.580894,1.996162,1.264709,2.283260


In [8]:
# 半年度
df5_1 = df2.copy(deep=True)
df5 = pd.DataFrame(columns=['Date']+df5_1.columns.to_list())
for year in range(start_year, 2022, 1):
    #
    df5_2 = df5_1[f'{year}-01-01': f'{year}-06-30'].iloc[[0, -1]].pct_change()*100
    df5.loc[df5.shape[0]+1] = [f'{year}-01-01 ~ {year}-06-30']+df5_2.iloc[-1].to_list()
    #
    df5_2 = df5_1[f'{year}-07-01': f'{year}-12-31'].iloc[[0, -1]].pct_change()*100
    df5.loc[df5.shape[0]+1] = [f'{year}-07-01 ~ {year}-12-31']+df5_2.iloc[-1].to_list()
print('半年度標準差(風險)')
print(df5.std())
print('半年度收益率(回報)')
df5

半年度標準差(風險)
hsi       13.590376
hsce      17.764275
ssec      24.987308
csi300    28.644640
sp500     10.519941
ixic      12.112991
shy        1.298064
tlt       11.206177
dtype: float64
半年度收益率(回報)


,Date,hsi,hsce,ssec,csi300,sp500,ixic,shy,tlt
1,2005-01-01 ~ 2005-06-30,1.104658,3.493590,-13.022163,-10.593064,0.276075,-2.414778,1.096993,12.109074
2,2005-07-01 ~ 2005-12-31,5.321349,9.225587,11.742925,8.682338,3.593395,6.088759,0.892718,-0.172595
3,2006-01-01 ~ 2006-06-30,7.023397,21.806319,41.597242,48.068997,-0.255996,-4.036735,0.771947,-6.613806
4,2006-07-01 ~ 2006-12-31,22.282944,52.241438,57.632859,43.702511,10.788251,10.265570,2.869981,7.830039
5,2007-01-01 ~ 2007-06-30,8.724593,15.978962,40.688447,82.095330,5.993627,6.105740,1.865438,-3.106101
6,2007-07-01 ~ 2007-12-31,23.562936,28.897132,34.921416,39.299551,-3.041571,1.115711,4.981016,11.774963
7,2008-01-01 ~ 2008-06-30,-19.805540,-25.595729,-48.109244,-48.156628,-11.550902,-12.133901,1.813638,-0.318992
8,2008-07-01 ~ 2008-12-31,-33.711838,-32.019517,-31.335075,-32.666913,-28.399868,-29.955227,4.546482,32.096384
9,2009-01-01 ~ 2009-06-30,18.090117,26.344661,57.352952,68.164787,-0.876598,12.715368,0.042737,-15.336011
10,2009-07-01 ~ 2009-12-31,18.255909,14.207944,6.612072,8.423759,25.657623,27.539911,0.557024,-2.322548
